In [2]:
import numpy as np
import glob

In [3]:
bbEmbeddingPath = "testrep/BubbleBobble"

In [4]:
columnRefArray = np.array([i+1 for i in range(13) for j in range(12)])

In [24]:
#bbEmbeddingPaths = sorted(glob.glob("testrep/BubbleBobble/level*.npy"))
bbEmbeddingPaths = sorted(glob.glob("/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/data/unified_rep/bubble_bobble/round*.pickle"))

In [27]:
import pickle

def get_pickle(pickle_path):
    '''
    This function loads the pickle file
    
    Input
    pickle_path: path of the pickle file
    Output
    Loads and returns the stored pickle file
    
    '''
    with open(pickle_path, "rb") as handle:
        level_pickle = pickle.load(handle)
    return level_pickle

In [28]:
N = 78

sosArray = np.ones(shape=(1, 256)) * 9
eosArray = np.ones(shape=(1, 256)) * 5

xTrain = []
xTrainHotColumnRef = []
yTrain = []
xTrainTargetIn = []

columnRef = []
for i, levelEmbeddingPath in enumerate(bbEmbeddingPaths):

    #levelEmbeddingArray = np.load(levelEmbeddingPath)
    levelEmbeddingArray = get_pickle(levelEmbeddingPath)

    for j in range(len(levelEmbeddingArray) - N):

        padLength = N - j

        dataI = np.concatenate((np.zeros(shape=(padLength, 256)), levelEmbeddingArray[:j]), axis=0)

        dataT = levelEmbeddingArray[j:j+N]
        targetIn = np.concatenate((sosArray, dataT))
        targetOut = np.concatenate((dataT, eosArray))

        levelIdx = np.concatenate((np.zeros(shape=(padLength)), columnRefArray[:j]), axis=0)
        dataC = np.zeros(shape=(N, 256))
        for j in range(N): dataC[j][int(levelIdx[j])] = 1

        columnRef.append(dataC)
        xTrain.append(dataI)
        xTrainTargetIn.append(targetIn)
        yTrain.append(targetOut)

xTrain = np.array(xTrain)
xTrainTargetIn = np.array(xTrainTargetIn)
yTrain = np.array(yTrain)
columnRef = np.array(columnRef)

In [29]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):

    def __init__(self, debug=False):
        super().__init__()

        self.histLSTM = nn.LSTM(78, 128)
        self.colLSTM = nn.LSTM(78, 128)

        self.textLSTM = nn.LSTM(79, 128)

        self.outputLayer = nn.Linear(128, 79)

    def forward(self, xHist, xText, xCol):
        
        histOut, (histH, histC) = self.histLSTM(xHist)

        colOut, (colH, colC) = self.colLSTM(xCol)

        hiddenAdd = torch.add(histH, histC)
        channelAdd = torch.add(colH, colC)

        textOut, (textH, textC) = self.textLSTM(xText, (hiddenAdd, channelAdd))

        output = nn.functional.tanh(self.outputLayer(textOut))
        
        return output

In [32]:
def TrainModel(xTrain, xTrainTargetIn, yTrain, columnRef, epochs, batchSize):

    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = LSTMModel()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001, eps=1e-7)

    criterion = nn.MSELoss()

    model.to(device)
    model.train()

    losses = []

    xTrain = xTrain.reshape(xTrain.shape[0], xTrain.shape[2], xTrain.shape[1])
    xTrainTargetIn = xTrainTargetIn.reshape(xTrainTargetIn.shape[0], xTrainTargetIn.shape[2], xTrainTargetIn.shape[1])
    yTrain = yTrain.reshape(yTrain.shape[0], yTrain.shape[2], yTrain.shape[1])
    columnRef = columnRef.reshape(columnRef.shape[0], columnRef.shape[2], columnRef.shape[1])

    for i in range(epochs):

        losses.append([])

        for j in range(0, xTrain.shape[0], batchSize):
            
            xTrainTensor = torch.tensor(xTrain[j:j+batchSize], dtype=torch.float32).to(device)
            xTrainTargetInTensor = torch.tensor(xTrainTargetIn[j:j+batchSize], dtype=torch.float32).to(device)

            yTrainTensor = torch.tensor(yTrain[j:j+batchSize], dtype=torch.float32).to(device)

            columnRefTensor = torch.tensor(columnRef[j:j+batchSize], dtype=torch.float32).to(device)

            # print(xTrainTensor.shape)
            # print(xTrainTargetInTensor.shape)
            # print(yTrainTensor.shape)
            # print(columnRefTensor.shape)

            yPred = model(xTrainTensor, xTrainTargetInTensor, columnRefTensor)
            
            loss = criterion(yPred, yTrainTensor)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            losses[i].append(loss.cpu().detach().item())

        print(f"Epoch {i}: loss {sum(losses[i])/len(losses[i])}")
    
    return model

In [33]:
TrainModel(xTrain, xTrainTargetIn, yTrain, columnRef, 5, 32)

Epoch 0: loss 0.4465213974231276
Epoch 1: loss 0.39754237838346385
Epoch 2: loss 0.39273248433039104
Epoch 3: loss 0.3900536111202733
Epoch 4: loss 0.38827900357287504


LSTMModel(
  (histLSTM): LSTM(78, 128)
  (colLSTM): LSTM(78, 128)
  (textLSTM): LSTM(79, 128)
  (outputLayer): Linear(in_features=128, out_features=79, bias=True)
)